<table align="left">
  <td>
    <a ref="C:/Users/josep/Snowman/Reforçat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" ref="C:/Users/josep/Snowman/Reforçat.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>



In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os
import glob
import random
import time
from IPython.display import clear_output


# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)


Intern code:

- 0 : out of grid and wall (x and #)
- 1 : small ball
- 2 : medium ball
- 3 : small ball on top of a medium ball
- 4 : large ball
- 5 : small ball on top of a large ball
- 6 : medium ball on top of a large ball
- 7 : small ball on top of a medium ball on top of a large ball
- 8 : grass (')
- 9 : snow (.)
- 10: character with snow on the floor (p)
- 11: character with grass on the floor (q)


In [ ]:
error=-20
tonto=-20
cami=0
camir=1
cim=3
cimbo=20
convertir=3
bingo=50

actions=[ 
        # pared                   bola petita          bola mitjana         bola pet s mit            bola grossa           bola pet s gr         bola mit s gr             tres boles            herba                   neu              
    [[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error]], #pared
    [[None,None,None,error],[None,None,None,tonto],['pq',11,3,cim]       ,[None,None,None,tonto],['pq',11,5,cim]       ,[None,None,None,tonto],['pq',11,7,bingo]     ,[None,None,None,tonto],['pq',11,1,camir]     ,['pq',11,2,convertir]],  #bola petita
    [[None,None,None,error],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],['pq',11,6,cimbo]     ,[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],['pq',11,2,camir]      ,['pq',11,4,convertir]],  #bola mitjana
    [[None,None,None,error],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,2,1,-cim]       ,[None,2,1,-cim]],        #bola pet s mit
    [[None,None,None,error],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],['pq',11,4,cami]      ,['pq',11,4,cami]],       #bola gran
    [[None,None,None,error],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,4,1,-cim]       ,[None,4,2,-cim]],        #bola pet s gr
    [[None,None,None,error],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,4,2,-cimbo]     ,[None,4,4,-cimbo]],      #bola mitj s gr
    [[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error]], #tres boles
    [['pq',11,None,cami]   ,['pq',11,None,cami]   ,['pq',11,None,cami]   ,['pq',11,None,cami]   ,['pq',11,None,cami]   ,['pq',11,None,cami]   ,['pq',11,None,cami]   ,['pq',11,None,tonto]  ,['pq',11,None,cami]   ,['pq',11,None,cami]],    #herba
    [['pq',10,None,cami]   ,['pq',10,None,cami]   ,['pq',10,None,cami]   ,['pq',10,None,cami]   ,['pq',10,None,cami]   ,['pq',10,None,cami]   ,['pq',10,None,cami]   ,['pq',10,None,tonto]  ,['pq',10,None,cami]   ,['pq',10,None,cami]]     #neu
] 


In [ ]:
path='C:/Users/josep/Snowman/Dades/'

files=glob.glob(path+"*.txt") 


n=8
m=16
tauler_base=np.zeros((n,m),dtype=int)




In [ ]:
ree="x#'.pq1234567"
reem=[0,0,8,9,10,11,1,2,3,4,5,6,7]
def lleguir_tauler(nom,n=n,m=m):
    f = open(nom, "r")
    tauler=tauler_base.copy()
    for row,linea in enumerate(f):
        linea=linea.rstrip('\n\r\t')
        for column,car in enumerate(linea):
            res = ree.find(car)
            tauler[row,column]=reem[res]
    f.close()
    res = np.array(np.where(tauler == 10))
    if np.size(res)!=0:
        tauler[res[0][0],res[1][0]]=9
    else:
        res = np.array(np.where(tauler == 11))
        tauler[res[0][0],res[1][0]]=8
    
    return np.array(tauler)

In [ ]:
def step(t,action,a,b): # 0 dreta, 1 baix, 2 esquerra, 3 dalt
    inc=[[0,1,0,-1],[1,0,-1,0]]
    seg=[a+inc[0][action],b+inc[1][action]]
    seg2=[a+2*inc[0][action],b+2*inc[1][action]]
    if seg[0]<8 and seg[1]<16 and seg2[0]<8 and seg2[1]<16:
        reward=actions[t[seg[0],seg[1]]][t[seg2[0],seg2[1]]][3]
    else:
        reward=error
    return (reward,seg[0],seg[1],seg2[0],seg2[1])

In [ ]:

def distancia(t,r,c,ball):
    dist=500
    for b in ball:
        res = np.array(np.where(t == b))
        if np.size(res)!=0:
            for j in range(res.shape[1]):
                if abs(r-res[0][j])+abs(c-res[1][j])<dist:
                    dist=abs(r-res[0][j])+abs(c-res[1][j])
    return dist
        
 

In [ ]:
def mod_step(t,reward,row,column,sega,segb,segaa,segbb):
    bonus=2
    if reward==cami:
        ball=[1,2]
        distact=distancia(t,row,column,ball)
        distnou=distancia(t,sega,segb,ball)
        if distnou<distact:
            reward=cami+bonus
    if reward==camir:
        if t[sega,segb]==1:
            ball=[2,6]
        elif t[sega,segb]==2:
            ball=[4]
        else:
            ball=[]
        distact=distancia(t,row,column,ball)
        distnou=distancia(t,sega,segb,ball)
        if distnou<distact:
            reward=camir+bonus
    if reward==convertir:
        if t[sega,segb]==2:
            res = np.isin(t,[4,5,6])
            res = np.array(np.where(res))
            if np.size(res)!=0:
                reward=error
        else:
            res = np.isin(t,[2,3,6])
            res = np.array(np.where(res))
            res2= np.isin(t,[4,5,6])
            res2= np.array(np.where(res2))
            if np.size(res)>0:
                if res.shape[1]>=2 or (res.shape[1]==1 and res2.shape[1]>=1):
                    reward=error
    
        
    return reward

        

In [ ]:



l=[]
r=[]
for f in files:
    fok=lleguir_tauler(f)
    for i in range(4):
        for row in range(len(fok)-1):
            for column in range(len(fok[row])-1):
                car=fok[row][column]
                if car==8 or car==9:
                    rewards=[0,0,0,0]
                    imp=False
                    for i in range(4): #per actions
                        tau=fok.copy()
                        tau[row][column]=10
                        reward,sega,segb,segaa,segbb=step(tau,i,row,column)
                        if tau[sega,segb]==1 and tau[segaa][segbb]==0:
                            imp=True
                        rewards[i]=mod_step(tau,reward,row,column,sega,segb,segaa,segbb)
                    '''if imp:
                        print('rewards',rewards)
                        print('eyes',np.eye(4)[np.argmax(rewards)])
                        aaa=input()'''
                    l.append(tau/10.)
                    r.append(np.eye(4)[np.argmax(rewards)])
        if i%2==0:
            fok=np.flip(fok,0)
        else:
            fok=np.flip(fok,1)
                    
                
    
    np.save(f+'t.npy',l)
    np.save(f+'r.npy',r)




In [ ]:
'''for f in [files[0]]:
    taulers=np.load(f+'t.npy','r')
    rewards=np.load(f+'r.npy','r')
    for i,t in enumerate(taulers):
        print(t)
        print(rewards[i])
        a=input()'''